In [41]:
import pandas as pd
import pyproj
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
import geopandas as gpd
import folium

# xlsx 파일에서 데이터 읽기
welfare_facility_df = pd.read_excel('kyeongdo_wido.xlsx')

# NaN 값을 포함하는 행 제거
welfare_facility_df = welfare_facility_df.dropna(subset=['위도', '경도'])

# 좌표계 변환을 위한 객체 생성
proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')
proj_local = pyproj.Proj('+proj=tmerc +lat_0=37 +lon_0=127 +k=1 +x_0=0 +y_0=0 +ellps=bessel +units=m +no_defs')


# 복지시설 위치를 중심으로 원형 폴리곤 생성
circle_list = []
for _, facility_row in welfare_facility_df.iterrows():
    lon, lat = facility_row['경도'], facility_row['위도']
    x, y = pyproj.transform(proj_wgs84, proj_local, lon, lat)
    circle = Point(x, y).buffer(250)  # 반경은 미터 단위입니다.
    circle_list.append(circle)

# 모든 원형 폴리곤을 하나의 GeoDataFrame으로 합치기
circle_gdf = gpd.GeoDataFrame({'geometry': circle_list}, crs='EPSG:5179')

# 모든 원형 폴리곤을 하나의 폴리곤으로 합치기
union_circle = circle_gdf.unary_union

# 여기서 union_circle이 MultiPolygon일 경우, 이를 GeoDataFrame으로 변환
if union_circle.geom_type == 'MultiPolygon':
    union_polygons = [polygon for polygon in union_circle.geoms]
    red_gdf = gpd.GeoDataFrame({'geometry': union_polygons}, crs='EPSG:5179')
else:
    red_gdf = gpd.GeoDataFrame({'geometry': [union_circle]}, crs='EPSG:5179')

# folium 지도 생성 (중심 좌표는 임의로 설정)
m = folium.Map(location=[37.5665, 126.9780], zoom_start=10)

# 합친 폴리곤을 지도에 그리기
for _, row in red_gdf.iterrows():
    polygon = row['geometry']
    lon, lat = pyproj.transform(proj_local, proj_wgs84, *polygon.exterior.coords.xy)
    coords = list(zip(lat, lon))
    folium.Polygon(locations=coords, color='blue', fill=True, fill_color='blue').add_to(m)

m

C:\Users\joon\AppData\Local\Temp\ipykernel_29596\1203519926.py:23: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x, y = pyproj.transform(proj_wgs84, proj_local, lon, lat)
C:\Users\joon\AppData\Local\Temp\ipykernel_29596\1203519926.py:46: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = pyproj.transform(proj_local, proj_wgs84, *polygon.exterior.coords.xy)


In [42]:
transformed_polygons = []
for _, row in red_gdf.iterrows():
    polygon = row['geometry']
    lon, lat = pyproj.transform(proj_local, proj_wgs84, *polygon.exterior.coords.xy)
    transformed_polygon = Polygon(zip(lon, lat))  # Create a Shapely Polygon with transformed coordinates
    transformed_polygons.append(transformed_polygon)

# Create a new GeoDataFrame with the transformed geometries
transformed_gdf = gpd.GeoDataFrame({'geometry': transformed_polygons}, crs='EPSG:4326')  # Assuming WGS84

# Save as GeoJSON file
transformed_gdf.to_file("transformed_polygons.geojson", driver='GeoJSON')


C:\Users\joon\AppData\Local\Temp\ipykernel_29596\2127084779.py:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = pyproj.transform(proj_local, proj_wgs84, *polygon.exterior.coords.xy)


In [43]:
# Read the saved GeoJSON file containing the transformed polygons
transformed_gdf = gpd.read_file("transformed_polygons.geojson")

# Create a Folium map
m = folium.Map(location=[37.5665, 126.9780], zoom_start=10)

# Iterate through the transformed GeoDataFrame to add polygons to the map
for _, row in transformed_gdf.iterrows():
    polygon = row['geometry']
    # Extract coordinates of the polygon
    lon, lat = zip(*polygon.exterior.coords)
    coords = list(zip(lat, lon))  # Reverse the order of lat, lon
    # Add polygon to the map
    folium.Polygon(locations=coords, color='blue', fill=True, fill_color='blue').add_to(m)

# Show the map
m